In [1]:
#spark streaming and batch processing
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [2]:
sc = SparkContext.getOrCreate()

In [3]:
df = sc.read("hdfs://localhost:54310/user/hadoopuser/Twitter/twitter_data.txt")

In [4]:
df.collect()

['{"created_at":"Fri May 29 16:01:22 +0000 2020","id":1266399242026917889,"id_str":"1266399242026917889","text":"Excited to share the latest addition to my #etsy shop: Vintage Inspired Art Deco 1.04ctw Emerald, Ruby and Sapphire\\u2026 https:\\/\\/t.co\\/FsrYk3PMRT","display_text_range":[0,140],"source":"\\u003ca href=\\"http:\\/\\/www.etsy.com\\" rel=\\"nofollow\\"\\u003eEtsy\\u003c\\/a\\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":259466947,"id_str":"259466947","name":"Tami Johnson","screen_name":"sobertj","location":"Mound, MN","url":null,"description":"Sober 11 Years, Working on MY Spirituality, Being More Creative","translator_type":"none","protected":false,"verified":false,"followers_count":14896,"friends_count":16387,"listed_count":103,"favourites_count":107800,"statuses_count":228572,"created_at":"Tue Mar 01 23:54:52 +0000 2011","utc_off

In [6]:
# Creating streaming context with 10 batches
ssc = StreamingContext(sc,10)

In [7]:
# checkpoint
ssc.checkpoint("checkpoint")


In [8]:
# streaming tweets from IP address
stream = ssc.socketTextStream('127.0.0.1',54310)

In [9]:
rows = stream.window(60)

In [10]:
hashtags_data= rows.flatMap(lambda text: text.splite(' ')).filter( lambda word:word.lower().startswith('#')).map( lambda word : (word.lower(),1)).reduceByKey(lambda a,b:a+b)

In [11]:
#Dstream mapping
dstream = hashtags_data.transform(lambda foo: foo.sortBy(lambda x:x[0].lower()).sortBy(lambda x:x[1],ascending=False))

In [12]:
#print dstrem
dstream.pprint()

In [14]:
# starting streaming session
import time
ssc.start()
time.sleep(5)

In [ ]:
ssc.awaitTermination()